In [ ]:
##### Fusion des fichier traiter précédent pour passer d'un fichier par comparaison à un fichier pour toute les comparaison
import pandas as pd
from os import listdir
from os.path import isfile, join
import json

myfiles2 = listdir("json-processed/")
main_df = pd.read_csv('json-processed/'+myfiles2[0], sep='\t', header=0)
for i in range(1, 10):
    df_temp = pd.read_csv('json-processed/'+myfiles2[i], sep='\t', header=0)
    main_df = main_df.merge(df_temp, on="9606", how="outer")
gene_names = main_df.copy()

In [ ]:
from Bio import Entrez
from Bio import SeqIO
import pandas as pd
Entrez.email = "corentin.meyer@etu.unistra.fr"
uni_gene_name = pd.read_csv("human_gene_name.txt", sep="\t")
species=["Homo sapiens", "Otolemur garnettii", "Chlorocebus sabaeus", "Nomascus leucogenys", "Callithrix jacchus",
         "Macaca fascicularis", "Macaca mulatta", "Papio anubis", "Gorilla gorilla gorilla",
          "Pan troglodytes", "Pongo abelii"]

##### Récupération du RefSeq ID à partir du Gene Name et de l'organisme d'intérêt
for i in uni_gene_name.iloc[:2,1]:
    #f = open(i+".id", "a"):
    for j in species:
        try:
            handle = Entrez.esearch(db="protein", term=i+" "+j, retmax=1)
            record = Entrez.read(handle)
            handle.close()
            #f.write(record["IdList"][0]+"\n")
            print(record)
        except:
            pass
    #f.close()

In [ ]:
term = uni_gene_name.iloc[:,1].tolist()

In [ ]:
import requests 
URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch"

##### Récupération du RefSeq ID à partir du Gene Name et de l'organisme d'intérêt
#for j in range(1):
term = uni_gene_name.iloc[:100,1].tolist()
term = [mygene + " " + species[j] for mygene in term]
myobj = {
    "retmax":1,
    "db":"protein",
    "term":"TONSL Homo sapiens"
}
x = requests.post(URL, data=myobj)
print(x.text)

In [ ]:
from Bio import Entrez
from Bio import SeqIO
import pandas as pd
Entrez.email = "corentin.meyer@etu.unistra.fr"
uni_gene_name = pd.read_csv("human_gene_name.txt", sep="\t")
species=["Homo sapiens", "Otolemur garnettii", "Chlorocebus sabaeus", "Nomascus leucogenys", "Callithrix jacchus",
         "Macaca fascicularis", "Macaca mulatta", "Papio anubis", "Gorilla gorilla gorilla",
          "Pan troglodytes", "Pongo abelii"]

##### Récupération du RefSeq ID à partir du Gene Name et de l'organisme d'intérêt
for j in species:
    print("Processing "+j)
    f = open("results/"+j+".id", "w")
    for i in uni_gene_name.iloc[:,1]:
        try:
            handle = Entrez.esearch(db="protein", term=i+" "+j, retmax=1)
            record = Entrez.read(handle)
            handle.close()
            f.write(record["IdList"][0]+"\n")
        except:
            f.write("Not Found - Error\n")
            pass
    f.close()

In [ ]:
import pandas as pd
from os import listdir
from os.path import isfile, join

myfiles2 = listdir("uniprot-mapping/")
main_df = pd.read_csv('json-processed/'+myfiles2[0]+".txt", sep='\t', header=0)
for i in range(1, 10):
    df_temp = pd.read_csv('json-processed/'+myfiles2[i]+".txt", sep='\t', header=0)
    main_df = main_df.merge(df_temp, on="9606", how="outer")
gene_names = main_df.copy()

In [ ]:
myfiles2.pop(11)
myfiles2.pop(11)
myfiles2

In [ ]:
for i in myfiles2:
    temp = pd.read_csv("uniprot-mapping/"+i, sep="\t")
    temp.columns = [i, i+"_RefSeq"]
    temp = temp.groupby([i]).agg(lambda col: ' '.join(col))
    temp = pd.DataFrame(temp)
    gene_names = gene_names.merge(temp, right_on=i, left_on=i, how="left")

In [ ]:
taxonID = [9606, 60711, 9541, 9544, 9555, 9595, 9598, 9601, 61853, 9483, 30611]
taxonID = [str(i)+"_RefSeq_x" for i in taxonID]
taxonID.insert(0, "9606")

In [ ]:
import math

taxonID = [9606, 60711, 9541, 9544, 9555, 9595, 9598, 9601, 61853, 9483, 30611]
taxonID = [str(i)+"_RefSeq_x" for i in taxonID]
taxonID.insert(0, "9606")
main_df = gene_names.iloc[:, [0, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]]
main_df = main_df[[i for i in taxonID]]

for i in range(20265):
    f = open("uniprot-mapping/results/"+str(gene_names.iloc[i,0])+".id", "w", newline='\n')
    for j in range(1, 12):
        query = str(main_df.iloc[i,j])
        if query=="nan":
            pass
        else:
            query = query.split(" ")
            for k in query:
                if k[0:2] == "NP":
                    f.write(str(k)+"\n")
                else:
                    f.write(str(k)+"\n")
    f.close()

In [ ]:
from Bio import Entrez
from Bio import SeqIO
import pandas as pd
Entrez.email = "corentin.meyer@etu.unistra.fr"
uni_gene_name = pd.read_csv("../refseq-long/Homo_sapiens.id", sep="\t", header=None)

##### Récupération du RefSeq ID à partir du Gene Name et de l'organisme d'intérêt
myFetchList =  uni_gene_name.iloc[:,0].tolist()
handle = Entrez.efetch(db="protein", id=myFetchList, retmax=len(myFetchList), rettype="fasta", retmode="text")
f = open("test.fasta", "w")
f.write(handle.read())
f.close()

In [ ]:
def fasta2List(pathFasta):
    f = open(pathFasta, "r")
    title = []
    seq = []
    seq_temp = []
    for line in f:
        if line[0] == ">":
            seq.append(''.join(seq_temp).replace("\n", ""))
            title.append(line.replace("\n", ""))
            seq_temp = []
        else:
            seq_temp.append(line)
    seq.append(''.join(seq_temp).replace("\n", ""))
    seq.pop(0)
    return [title, seq]

def FindNotFound(pathID):
    f = open(pathID, "r")
    myID = []
    IDfile = pd.read_csv(pathID, sep="\t", header=None)
    geneListID = IDfile.iloc[:,0].tolist()
    for i, j in enumerate(geneListID):
        if j == 'Not Found - Error':
            myID.append(i)
    return myID

def AlterFasta(fastaPath, idPath):
    t, s = fasta2List(fastaPath)
    error = FindNotFound(idPath)
    for i in error:
        t.insert(i, ">NaN")
        s.insert(i, "XXX")
    f = open(fastaPath+".modified", "w")
    for i in range(len(t)):
        f.write(t[i]+"\n")
        f.write(s[i]+"\n\n")
    f.close()

In [ ]:
# RefSeq-Long -> Line per file
fullTable = pd.read_csv("../refseq-long/fulltable.csv", sep="\t", header=None)
fullTable.columns = ["Homo", "Callithrix", "Gorilla", "Macaca1", "Pan", "Chlorocebus", "Nomascus", "Papio", "Macaca2", "Otolemu", "Pongo"]

for i in range(20071):
    f = open("../refseq-long/results/"+str(fullTable.iloc[i,0])+".id", "w", newline='\n')
    query = fullTable.iloc[i,:].tolist()
    for i in query:
        f.write(str(i)+"\n")
    f.close()

In [ ]:
homo = fullTable.iloc[:,0]